# Customer Lifetime Value Prediction

Marketing Model to predict the customer lifetime value for Olist, a Brazilian E-Commerce Company.

Predicting the lifetime value for a company will allow stakeholders to make key decisions on when, how, and where money should be invested. With my analysis, stakeholders will be able to identify customers based on their most recent orders, most frequent, and the average monetary value; otherwise known as the RFM Matrix.

In the analysis, we will need to work on the features that will be fed into the model. Clusters will be created based on the features of the RFM matrix, which will be used to segment customers.

Machine Learning models used were the Logistics Regression for multi-class classification and then Linear Regression to predict the monetary value of the customers.

# Imports

In [1]:
# dataframe libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

# Data Plotting Libraries
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Pre-Processing Libraries
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# ML Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Metric Libraries
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score

# etc
import warnings
warnings.filterwarnings('ignore')
import pickle

# Reading data into their respective dataframes

In [2]:
# Reading all files into df
customers = pd.read_csv('files_for_project/olist_customers_dataset.csv') # Main df to read all information into
orders = pd.read_csv('files_for_project/olist_orders_dataset.csv') # To get information about orders
payments = pd.read_csv('files_for_project/olist_order_payments_dataset.csv') # To get payment information

# Merging orders and payments to get orderid, count of orders, and ttl revenue
ordersandpayments = pd.merge(orders, payments , on = ['order_id'])

# Creating pivot
oandppivot = ordersandpayments.pivot_table(index = ['customer_id','order_id','order_delivered_customer_date'], 
                                           values = ['payment_value'], aggfunc = ['count','sum'])

# Setting column names
oandppivot.columns = ('orders', 'ttl_revenue') 

# resetting index
oandppivot = oandppivot.reset_index()

# merging oandppivot with customer information to get customer_id, num_of_orders, and ttl_revenue
custorder = pd.merge(customers, oandppivot, on = 'customer_id')

# Creating final df
final_df = custorder.pivot_table(index = ['customer_unique_id',
                                          'order_id',
                                          'order_delivered_customer_date',
                                          'customer_zip_code_prefix',
                                          'customer_city',
                                          'customer_state'], 
                                 values = ['ttl_revenue'], 
                                 aggfunc = ['count', 'sum'])

# Resetting df index
final_df = final_df.reset_index()

# Renaming column headers
final_df.columns = ('customer_id','order_id','order_date','zip_code','city','state','ttl_order','ttl_revenue')

# df check
final_df

,customer_id,order_id,order_date,zip_code,city,state,ttl_order,ttl_revenue
0,0000366f3b9a7992bf8c76cfdf3221e2,e22acc9c116caa3f2b7121bbb380d08e,2018-05-16 20:48:37,7787,cajamar,SP,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,3594e05a005ac4d06a72673270ef9ec9,2018-05-10 18:02:42,6053,osasco,SP,1,27.19
2,0000f46a3911fa3c0805444483337064,b33ec3b699337181488304f362a6b734,2017-04-05 14:38:47,88115,sao jose,SC,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,41272756ecddd9a9ed0180413cc22fb6,2017-11-01 21:23:05,66812,belem,PA,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,d957021f1127559cd947b62533f484f7,2017-11-27 23:08:56,18040,sorocaba,SP,1,196.89
...,...,...,...,...,...,...,...,...
96470,fffcf5a5ff07b0908bd4e2dbc735a684,725cf8e9c24e679a8a5a32cb92c9ce1e,2017-07-06 09:22:00,55250,sanharo,PE,1,2067.42
96471,fffea47cd6d3cc0a88bd621562a9d061,c71b9252fd7b3b263aaa4cb09319a323,2018-01-09 22:28:20,44054,feira de santana,BA,1,84.58
96472,ffff371b4d645b6ecea244b27531430a,fdc45e6c7555e6cb3cc0daca2557dbe1,2017-02-22 12:45:04,78552,sinop,MT,1,112.46
96473,ffff5962728ec6157033ef9805bacc48,94d3ee0bc2a0af9d4fa47a4d63616e8d,2018-05-14 11:54:26,29460,bom jesus do norte,ES,1,133.69


In [3]:
# Converting order_date column to DateTime format
final_df['order_date'] = pd.to_datetime(final_df['order_date']) 

# Checking to make sure order_date has been converted
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96475 entries, 0 to 96474
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   customer_id  96475 non-null  object        
 1   order_id     96475 non-null  object        
 2   order_date   96475 non-null  datetime64[ns]
 3   zip_code     96475 non-null  int64         
 4   city         96475 non-null  object        
 5   state        96475 non-null  object        
 6   ttl_order    96475 non-null  int64         
 7   ttl_revenue  96475 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 5.9+ MB


# Sorting Through DataFrame

In [4]:
# Creating df for unique customers and their most recent purhcase
last_purchase = final_df.groupby('customer_id').order_date.max().reset_index()
last_purchase.columns = ['customer_id', 'last_purchase_date']
last_purchase['recency'] = (last_purchase['last_purchase_date'].max() - last_purchase['last_purchase_date']).dt.days

# Creating recency df
user_rec = last_purchase.drop(['last_purchase_date'], axis = 1)

# Creating column for ttl_revenue
user_rev = final_df.groupby('customer_id')['ttl_revenue'].sum().sort_values(ascending = False).reset_index()
user_rev.columns = ('customer_id','ttl_revenue')

# Checking on frequency
user_freq = final_df.groupby('customer_id')['ttl_order'].sum().sort_values(ascending = False).reset_index()
user_freq.columns = ('customer_id', 'ttl_orders')

# Merging all dataframes
merge_1 = pd.merge(user_rec, user_freq, on = 'customer_id')
users_df = pd.merge(merge_1, user_rev, on = 'customer_id')

# df check
users_df



,customer_id,recency,ttl_orders,ttl_revenue
0,0000366f3b9a7992bf8c76cfdf3221e2,153,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,159,1,27.19
2,0000f46a3911fa3c0805444483337064,559,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,349,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,323,1,196.89
...,...,...,...,...
93350,fffcf5a5ff07b0908bd4e2dbc735a684,468,1,2067.42
93351,fffea47cd6d3cc0a88bd621562a9d061,280,1,84.58
93352,ffff371b4d645b6ecea244b27531430a,602,1,112.46
93353,ffff5962728ec6157033ef9805bacc48,156,1,133.69


In [5]:
# Scaling recency, ttl_orders, and ttl_revenue to help with KMeans scoring
users_num = users_df.select_dtypes(include = 'number')

# Running Scaler
users_scale = StandardScaler().fit_transform(users_num)
users_scale = pd.DataFrame(users_scale, columns = users_num.columns)
users_scale

,recency,ttl_orders,ttl_revenue
0,-0.790621,-0.159832,-0.102934
1,-0.751144,-0.159832,-0.609793
2,1.880653,-0.159832,-0.348962
3,0.498960,-0.159832,-0.537195
4,0.327893,-0.159832,0.140045
...,...,...,...
93350,1.281919,-0.159832,8.405185
93351,0.044974,-0.159832,-0.356209
93352,2.163571,-0.159832,-0.233018
93353,-0.770883,-0.159832,-0.139211


# KMeans Clustering / Customer Segmentation

In [6]:
# Building clusters and adding clusters to user_df dataframe

kmeans = KMeans(n_clusters = 5, random_state = 42)
users_scale['reccluster'] = kmeans.fit_predict(users_scale[['recency']])
users_scale['freqcluster'] = kmeans.fit_predict(users_scale[['ttl_orders']])
users_scale['revcluster'] = kmeans.fit_predict(users_scale[['ttl_revenue']])

In [7]:
# Creating function to group clusters correctly based on their grouping

def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final



In [8]:
# running function on user_df for cluster and rececy
users_final = order_cluster('reccluster', 'recency',users_scale,False)
users_final = order_cluster('freqcluster', 'ttl_orders',users_scale,False)
users_final = order_cluster('revcluster', 'ttl_revenue',users_scale,False)

In [9]:
# Applying ordered clusters to original dataframe
clusters = users_final.drop(['recency','ttl_orders','ttl_revenue'], axis = 1)
users_clusters = users_df.join(clusters)

# Creating an overall score for the clusters
users_clusters['overallscore'] = users_clusters['reccluster'] + users_clusters['freqcluster'] + users_clusters['revcluster']

# Customer Segmentation based on overall score (Will be done in 3 segments)
users_clusters['segment'] = 'Low'
users_clusters.loc[users_clusters['overallscore'] > 3, 'segment'] = 'Mid'
users_clusters.loc[users_clusters['overallscore'] > 7, 'segment'] = 'High'
users_clusters

,customer_id,recency,ttl_orders,ttl_revenue,reccluster,freqcluster,revcluster,overallscore,segment
0,0000366f3b9a7992bf8c76cfdf3221e2,153,1,141.90,4,0,4,8,High
1,0000b849f77a49e4a4ce2b2a4ca5be3f,159,1,27.19,4,0,4,8,High
2,0000f46a3911fa3c0805444483337064,559,1,86.22,1,0,4,5,Mid
3,0000f6ccb0745a6a4b88665a16c9f078,349,1,43.62,0,0,4,4,Mid
4,0004aac84e0df4da2b147fca70cf8255,323,1,196.89,4,0,4,8,High
...,...,...,...,...,...,...,...,...,...
93350,fffcf5a5ff07b0908bd4e2dbc735a684,468,1,2067.42,4,0,0,4,Mid
93351,fffea47cd6d3cc0a88bd621562a9d061,280,1,84.58,0,0,0,0,Low
93352,ffff371b4d645b6ecea244b27531430a,602,1,112.46,1,0,0,1,Low
93353,ffff5962728ec6157033ef9805bacc48,156,1,133.69,1,0,0,1,Low


# Customer Lifetime Value (CLTV) Calculation

In [10]:
# Segmenting Customers Based on time segments 6 months and one year

# Past 6 month users
users_6mnth = final_df[(final_df.order_date < pd.datetime(2018, 10, 1)) & 
                       (final_df.order_date >= pd.datetime(2018, 4, 1))].reset_index(drop = True)


# Past year customers
users_12mnth = final_df[(final_df.order_date < pd.datetime(2018, 10, 1)) & 
                       (final_df.order_date >= pd.datetime(2017, 10, 1))].reset_index(drop = True)

# Six Month Churn Calculation

In [11]:
# DataFrame for revenue per customer_id
user6m = pd.DataFrame(users_6mnth.groupby('customer_id')['ttl_revenue'].sum().reset_index())
user6m.columns = ['customer_id','6m_rev']

# Combining 6 month users + orginal dataframe
merge6mnth = pd.merge(users_clusters, user6m, on = 'customer_id', how = 'right') # using 'right' to provide for only customers in 6 month category

# Creating clusters based on segment 
kmeans = KMeans(n_clusters = 3, random_state = 42)
merge6mnth['LTV6'] = kmeans.fit_predict(merge6mnth[['6m_rev']])

# Ordering LTV cluster by relevance
LTV_cluster6 = order_cluster('LTV6', '6m_rev', merge6mnth, True)


In [12]:
# Need to dummify categorical column 'segment'
LTV_cluster6 = pd.get_dummies(data = LTV_cluster6, columns = ['segment'])

# Logistic Regression Prediction - Six Months

In [13]:
# Splitting X and Y

X = LTV_cluster6.drop(['customer_id','LTV6'], axis = 1)
y = LTV_cluster6['LTV6']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [14]:
# Running Logistics classifier

LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

LR_model = LR.fit(X_train, y_train) 

LR_predict = LR.predict(X_test)

print('The Logistic Regression accuracy score: ', round(LR.score(X_train,y_train), 4)) # Accuracy Score
print()

# Classification Score
print('Classification Report: ')
print()
print(classification_report(y_test, LR_predict)) 

# Checking for overfitting or underfitting with cross_val score

scores = cross_val_score(LR_model, X_train, y_train, cv=10)
print("Checking for overfitting / underfitting: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

The Logistic Regression accuracy score:  0.9758

Classification Report: 

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      9578
           1       0.86      0.85      0.86       961
           2       0.73      0.80      0.76        88

    accuracy                           0.97     10627
   macro avg       0.86      0.88      0.87     10627
weighted avg       0.97      0.97      0.97     10627

Checking for overfitting / underfitting: 0.98 (+/- 0.01)


# Twelve Year Churn Calculation

In [15]:
# DataFrame for revenue per customer_id
user12m = pd.DataFrame(users_12mnth.groupby('customer_id')['ttl_revenue'].sum().reset_index())
user12m.columns = ['customer_id','12m_rev']

# Combining 6 month users + orginal dataframe
merge12mnth = pd.merge(users_clusters, user12m, on = 'customer_id', how = 'right') # using 'right' to provide for only customers in 6 month category

# Creating clusters based on segment 
kmeans = KMeans(n_clusters = 3, random_state = 42)
merge12mnth['LTV12'] = kmeans.fit_predict(merge12mnth[['12m_rev']])

# Ordering LTV cluster by relevance
LTV_cluster12 = order_cluster('LTV12', '12m_rev', merge12mnth, True)


In [16]:
# Dummifying Columns
LTV_cluster12 = pd.get_dummies(data = LTV_cluster12, columns = ['segment'])

# Logisitc Regression Prediction - Twelve Months

In [17]:
# Splitting X and Y

X = LTV_cluster12.drop(['customer_id','LTV12'], axis = 1)
y = LTV_cluster12['LTV12']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [18]:

LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

LR_model = LR.fit(X_train, y_train) 

LR_predict = LR.predict(X_test)

print('The Logistic Regression accuracy score: ', round(LR.score(X_train,y_train), 4)) # Accuracy Score
print()

# Classification Score
print('Classification Report: ')
print()
print(classification_report(y_test, LR_predict)) 

# Checking for overfitting or underfitting with cross_val score

scores = cross_val_score(LR_model, X_train, y_train, cv=10)
print("Checking for overfitting / underfitting: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

The Logistic Regression accuracy score:  0.9747

Classification Report: 

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18628
           1       0.87      0.86      0.87      2105
           2       0.72      0.74      0.73       209

    accuracy                           0.97     20942
   macro avg       0.86      0.86      0.86     20942
weighted avg       0.97      0.97      0.97     20942

Checking for overfitting / underfitting: 0.97 (+/- 0.01)


# Exporting File

In [63]:
# Creating df to retrieve LTV clusters and Revenue for both 6 month and 12 month churns
ltv12 = pd.DataFrame(LTV_cluster12[['customer_id','12m_rev','LTV12']])
ltv6 = pd.DataFrame(LTV_cluster6[['customer_id','6m_rev','LTV6']])
ltv6merge = pd.merge(ltv6, users_clusters, on = 'customer_id', how = 'right')
final_export = pd.merge(ltv12, ltv6merge, on = 'customer_id', how = 'right' )

final_export.to_csv('files_for_project/final_export.csv')
